In [ ]:
import cv2
import os
import numpy as np
import random

input_root = "dataset"
output_root = "augmented_dataset"

# Create main output directory
os.makedirs(output_root, exist_ok=True)

def rotate_image(img, angle):
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1)
    return cv2.warpAffine(img, M, (w, h))

def add_gaussian_noise(img):
    mean = 0
    std = 25      # adjust noise level if needed
    noise = np.random.normal(mean, std, img.shape).astype(np.uint8)
    noisy_img = cv2.add(img, noise)
    return noisy_img

# Loop over classes
for class_name in os.listdir(input_root):
    class_path = os.path.join(input_root, class_name)
    if not os.path.isdir(class_path):
        continue

    # Create class folder inside augmented_dataset
    output_class_path = os.path.join(output_root, class_name)
    os.makedirs(output_class_path, exist_ok=True)

    # Loop over images
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path)

        if img is None:
            continue

        filename = os.path.splitext(img_name)[0]

        # ----- Augmentations -----
        # 1. Flip
        flipped = cv2.flip(img, 1)
        cv2.imwrite(os.path.join(output_class_path, f"{filename}_flip.png"), flipped)

        # 2. Rotation
        angle = random.randint(-30, 30)  # rotate between -30 to +30 degrees
        rotated = rotate_image(img, angle)
        cv2.imwrite(os.path.join(output_class_path, f"{filename}_rot.png"), rotated)

        # 3. Gaussian Noise
        noisy = add_gaussian_noise(img)
        cv2.imwrite(os.path.join(output_class_path, f"{filename}_noise.png"), noisy)

print("Augmentation completed successfully!")
